In [1]:
# Mount your google drive so we have access to the models and assets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! git clone https://github.com/huggingface/diffusers

Cloning into 'diffusers'...
remote: Enumerating objects: 47722, done.
remote: Counting objects: 100% (1551/1551), done.
remote: Compressing objects: 100% (695/695), done.
remote: Total 47722 (delta 1011), reused 1150 (delta 727), pack-reused 46171
Receiving objects: 100% (47722/47722), 31.86 MiB | 22.38 MiB/s, done.
Resolving deltas: 100% (35191/35191), done.


In [3]:
!cd diffusers && pip install .

Processing /content/diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.25.0.dev0-py3-none-any.whl size=1816331 sha256=0cc07a0661f01f4ca7cfabd8a1e667219d45b17cba640fa94a0b1eeac0f762f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-ow9pz890/wheels/95/c5/3b/e1b4269f8a2584de57e75f949a185b48fc4144e9a91fc9965a
Successfully built diffusers


In [4]:
!cd diffusers/examples/text_to_image && pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00


In [5]:
from accelerate.utils import write_basic_config

write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [6]:
!cat /root/.cache/huggingface/accelerate/default_config.yaml

{
  "compute_environment": "LOCAL_MACHINE",
  "debug": false,
  "distributed_type": "NO",
  "downcast_bf16": false,
  "machine_rank": 0,
  "main_training_function": "main",
  "mixed_precision": "no",
  "num_machines": 1,
  "num_processes": 1,
  "rdzv_backend": "static",
  "same_network": false,
  "tpu_use_cluster": false,
  "tpu_use_sudo": false,
  "use_cpu": false
}


In [7]:
%cd diffusers/examples/text_to_image/

/content/diffusers/examples/text_to_image


In [10]:
! export MODEL_NAME="runwayml/stable-diffusion-v1-5" && \
export OUTPUT_DIR="/content/drive/MyDrive/pokemon-lora" && \
export HUB_MODEL_ID="username/pokemon-lora" && \
export DATASET_NAME="diffusers/pokemon-gpt4-captions" && \
accelerate launch train_text_to_image_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$DATASET_NAME \
  --dataloader_num_workers=0 \
  --hub_token="huggingface_read_token" \
  --resolution=512 \
  --center_crop \
  --random_flip \
  --train_batch_size=4 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=5500 \
  --learning_rate=1e-04 \
  --max_grad_norm=1 \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=0 \
  --output_dir=${OUTPUT_DIR} \
  --push_to_hub \
  --hub_model_id=${HUB_MODEL_ID} \
  --report_to=tensorboard \
  --checkpointing_steps=500 \
  --resume_from_checkpoint='latest' \
  --validation_prompt="A pokemon with blue eyes." \
  --seed=918

2023-12-13 18:43:44.299111: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 18:43:44.299171: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 18:43:44.302351: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 18:43:46.763424: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
12/13/2023 18:43:48 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

{'thresholding', 'variance_type', 'prediction_type', 'dynamic_thresholdin

## Upload Model to Hugging Face
Note that the authorization token on hugging face is always different for read and write operations.

In [24]:
from huggingface_hub import HfApi
api = HfApi()

hub_token="huggingface_write_token"
repo_id="username/pokemon-lora"

repourl = api.create_repo(repo_id,
                          token=hub_token,
                          exist_ok=True)

In [30]:
OUTPUT_DIR = "/content/drive/MyDrive/pokemon-lora"
api.upload_folder(repo_id=repo_id,
                  folder_path=OUTPUT_DIR,
                  commit_message="Finished Finetuning",
                  token=hub_token,
                  ignore_patterns=["step_*", "epoch_*"])

pytorch_lora_weights.safetensors:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

random_states_0.pkl:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

Upload 28 LFS files:   0%|          | 0/28 [00:00<?, ?it/s]

scheduler.bin:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

optimizer.bin:   0%|          | 0.00/6.59M [00:00<?, ?B/s]

...

events.out.tfevents.1702475841.2a75ee7015a3.3818.0:   0%|          | 0.00/35.5M [00:00<?, ?B/s]

events.out.tfevents.1702483097.2a75ee7015a3.34281.0:   0%|          | 0.00/38.3M [00:00<?, ?B/s]

events.out.tfevents.1702493063.2a75ee7015a3.75301.0:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

pytorch_lora_weights.safetensors:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

'https://huggingface.co/bryant0918/pokemon-lora/tree/main/'

# Inference

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")
pipeline.load_lora_weights("/content/drive/MyDrive/pokemon-lora/checkpoint-5500", weight_name="pytorch_lora_weights.safetensors")
# Disable Safety Checker
pipeline.safety_checker = None
pipeline.requires_safety_checker = False

In [ ]:
from IPython.display import display
image = pipeline("A christmas pokemon with green eyes").images[0]
display(image)

In [ ]:
image = pipeline("A santa pokemon doing karate").images[0]
display(image)

In [ ]:
image = pipeline("A robot pokemon surfing").images[0]
display(image)

In [ ]:
image = pipeline("An astronaut pokemon in space.").images[0]
display(image)

In [ ]:
image = pipeline("A pokemon on Mars.").images[0]
display(image)

In [ ]:
image = pipeline("A dragon pokemon in front of a red barn.").images[0]
display(image)